In [62]:
import itertools as it
import random

import einops
import torch as t
import torch.nn.functional as F
from torch.optim.lr_scheduler import LRScheduler, ReduceLROnPlateau
from tqdm import tqdm

from src.dataset import Dataset

In [5]:

class ContrastiveCNN(t.nn.Module):
    """Simple 2-layer CNN for MNIST"""

    def __init__(self, d_embed: int = 64) -> None:
        super().__init__()
        self.d_embed = d_embed
        
        CONV_KERNEL_SIZE = 5
        CONV_STRIDE = 1
        POOL_KERNEL_SIZE = 2
        POOL_STRIDE = 2

        self.conv1 = t.nn.Conv2d(
            in_channels=1,
            out_channels=32,
            kernel_size=CONV_KERNEL_SIZE,
            stride=CONV_STRIDE,
            padding="same",
        )
        self.pool1 = t.nn.MaxPool2d(kernel_size=POOL_KERNEL_SIZE, stride=POOL_STRIDE)
        self.conv2 = t.nn.Conv2d(
            in_channels=32,
            out_channels=64,
            kernel_size=CONV_KERNEL_SIZE,
            stride=CONV_STRIDE,
            padding="same",
        )
        self.pool2 = t.nn.MaxPool2d(kernel_size=POOL_KERNEL_SIZE, stride=POOL_STRIDE)
        self.fc = t.nn.Linear(in_features=7 * 7 * 64, out_features=d_embed)

    def forward(self, x: t.Tensor) -> t.Tensor:  # [batch 28 28]
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = x.flatten(1)
        x = self.fc(x)
        return x



In [32]:
def contrastive_loss(
    embeds: t.Tensor, # [batch d_embed]
    labels: t.Tensor, # [batch (d_label=10)]
    *,
    epsilon: float = 1e-4 
) -> t.Tensor:
    assert embeds.ndim == 2
    assert labels.ndim == 1
    assert len(embeds) == len(labels)
    labels_rep = einops.repeat(labels, "b -> b rep", rep=len(labels))
    diff_pair_inds = (labels_rep == labels_rep.T).nonzero()
    same_pair_inds = (labels_rep != labels_rep.T).nonzero()
    #TODO: figure out how to this optimally, with tensors, einops, gather, etc, rather than loops
    #TODO: also, better variable names
    diff_pair_diffs_squared = t.stack(
        [
            (embeds[i] - embeds[j]) ** 2
            for i, j in diff_pair_inds
        ]
    )
    same_pair_diffs = t.stack(
        [
            (embeds[i] - embeds[j])
            for i, j in same_pair_inds
        ]
    )
    
    loss = diff_pair_diffs_squared.mean()
    return loss


In [37]:

def train(
    model: ContrastiveCNN,
    ds: Dataset,
    optimizer: t.optim.Optimizer,
    *,
    n_epochs: int = 10,
    batch_size: int = 32,
    verbose: bool = True,
    scheduler: LRScheduler | ReduceLROnPlateau | None = None
) -> None: # TrainingHistory:
    for epoch_i in range(1, n_epochs + 1):
        batch_gen = ds.get_train_batch_gen(batch_size)
        batch_losses: list[float] = []
        
        for batch_x, batch_y in batch_gen:
            optimizer.zero_grad()
            batch_embeds = model(batch_x)
            batch_loss = contrastive_loss(batch_embeds, batch_y)
            batch_loss.backward()
            optimizer.step()
            batch_losses.append(batch_loss.item())
        
        epoch_loss = t.tensor(batch_losses).mean()
        
        if isinstance(scheduler, LRScheduler):
            scheduler.step()
        elif isinstance(scheduler, ReduceLROnPlateau):
            scheduler.step(epoch_loss)
        
        if verbose:
            print(f"[{epoch_i}] {epoch_loss = }")
            if epoch_loss == 0:
                print(f"Achieved zero loss, terminating training after {epoch_i} epochs")
                break

In [38]:
model = ContrastiveCNN()
ds = Dataset.load()
lr = 1e-3
optimizer = t.optim.AdamW(model.parameters(), lr)
scheduler = ReduceLROnPlateau(optimizer)
train(model, ds, optimizer, scheduler=scheduler)


[1] epoch_loss = tensor(9.8985e-06)
[2] epoch_loss = tensor(8.7086e-12)
[3] epoch_loss = tensor(3.8355e-13)
[4] epoch_loss = tensor(1.6754e-14)
[5] epoch_loss = tensor(4.7133e-16)
[6] epoch_loss = tensor(0.)
Achieved zero loss, terminating training after 6 epochs


## Testing

In [60]:
test_embeds = model(ds.test_x)
labels = list(range(10))
embed_dists = t.zeros(10, 10, requires_grad=False)

with t.no_grad():
    for label1, label2 in tqdm(it.product(labels, repeat=2)):
        embeds1_mean = test_embeds[ds.test_y == label1].mean(0)
        embeds2_mean = test_embeds[ds.test_y == label2].mean(0)
        embed_dist = ((embeds1_mean - embeds2_mean) ** 2).mean()
        embed_dists[label1, label2] = embed_dist
    
# Normalize so that the highest value is one
embed_dists *= 1 / embed_dists.max()

100it [00:00, 6898.75it/s]


In [61]:
from plotly import express as px, graph_objects as go

fig = px.imshow(embed_dists)
fig.show()

In [83]:
all_train_embeds = model(ds.train_x)
all_test_embeds = model(ds.test_x)

In [115]:
model(ds.train_x[:1])

tensor([[ 0.0048,  0.0122,  0.0070,  0.0141,  0.0066, -0.0043, -0.0110, -0.0130,
          0.0053,  0.0087, -0.0064, -0.0049,  0.0008, -0.0153, -0.0101, -0.0133,
          0.0066,  0.0019, -0.0004, -0.0066, -0.0157, -0.0006, -0.0031, -0.0074,
         -0.0059,  0.0012,  0.0101,  0.0154, -0.0008,  0.0020, -0.0039,  0.0136,
         -0.0120, -0.0157, -0.0066,  0.0029,  0.0040,  0.0157,  0.0086,  0.0022,
         -0.0093, -0.0106, -0.0134, -0.0057,  0.0021,  0.0113, -0.0017, -0.0148,
         -0.0060, -0.0138,  0.0054, -0.0019, -0.0069,  0.0096, -0.0068,  0.0013,
         -0.0032, -0.0052,  0.0035,  0.0094, -0.0027,  0.0036,  0.0142,  0.0054]],
       grad_fn=<AddmmBackward0>)

In [116]:
model(ds.train_x[1:2])

tensor([[ 0.0048,  0.0122,  0.0070,  0.0141,  0.0066, -0.0043, -0.0110, -0.0130,
          0.0053,  0.0087, -0.0064, -0.0049,  0.0008, -0.0153, -0.0101, -0.0133,
          0.0066,  0.0019, -0.0004, -0.0066, -0.0157, -0.0006, -0.0031, -0.0074,
         -0.0059,  0.0012,  0.0101,  0.0154, -0.0008,  0.0020, -0.0039,  0.0136,
         -0.0120, -0.0157, -0.0066,  0.0029,  0.0040,  0.0157,  0.0086,  0.0022,
         -0.0093, -0.0106, -0.0134, -0.0057,  0.0021,  0.0113, -0.0017, -0.0148,
         -0.0060, -0.0138,  0.0054, -0.0019, -0.0069,  0.0096, -0.0068,  0.0013,
         -0.0032, -0.0052,  0.0035,  0.0094, -0.0027,  0.0036,  0.0142,  0.0054]],
       grad_fn=<AddmmBackward0>)

In [111]:
SEED = 42
random.seed(SEED)
t.manual_seed(SEED)

# Take 10 random examples from training set for each label to make label representations
label_embeds = t.zeros(10, model.d_embed)
with t.no_grad():
    for label in labels:
        train_x_index = random.sample((ds.train_y == label).nonzero().tolist(), k=10)
        stacked_embeds = t.stack([all_train_embeds[i] for i in train_x_index]).squeeze()
        print(stacked_embeds)
        
        label_embed = stacked_embeds.mean(0)
        # print(label, label_embed)
        label_embeds[label, :] = label_embed

tensor([[ 0.0048,  0.0122,  0.0070,  0.0141,  0.0066, -0.0043, -0.0110, -0.0130,
          0.0053,  0.0087, -0.0064, -0.0049,  0.0008, -0.0153, -0.0101, -0.0133,
          0.0066,  0.0019, -0.0004, -0.0066, -0.0157, -0.0006, -0.0031, -0.0074,
         -0.0059,  0.0012,  0.0101,  0.0154, -0.0008,  0.0020, -0.0039,  0.0136,
         -0.0120, -0.0157, -0.0066,  0.0029,  0.0040,  0.0157,  0.0086,  0.0022,
         -0.0093, -0.0106, -0.0134, -0.0057,  0.0021,  0.0113, -0.0017, -0.0148,
         -0.0060, -0.0138,  0.0054, -0.0019, -0.0069,  0.0096, -0.0068,  0.0013,
         -0.0032, -0.0052,  0.0035,  0.0094, -0.0027,  0.0036,  0.0142,  0.0054],
        [ 0.0048,  0.0122,  0.0070,  0.0141,  0.0066, -0.0043, -0.0110, -0.0130,
          0.0053,  0.0087, -0.0064, -0.0049,  0.0008, -0.0153, -0.0101, -0.0133,
          0.0066,  0.0019, -0.0004, -0.0066, -0.0157, -0.0006, -0.0031, -0.0074,
         -0.0059,  0.0012,  0.0101,  0.0154, -0.0008,  0.0020, -0.0039,  0.0136,
         -0.0120, -0.0157, 

In [109]:
label_embeds[:,2]

tensor([0.0070, 0.0070, 0.0070, 0.0070, 0.0070, 0.0070, 0.0070, 0.0070, 0.0070,
        0.0070])

In [103]:
label_embeds[0] == label_embeds[1]

tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True])

In [99]:
def predict_classes(
    x: t.Tensor # [batch 1 28 28]
) -> t.Tensor: # [batch] (ints of classes 0:9)
    with t.no_grad():
        x_embeds = model(x)
        dists = t.stack([((x_embeds - label_embed) ** 2).mean() for label_embed in label_embeds])
    print(dists)
predict_classes(ds.test_x[:1])

tensor([3.3089e-19, 3.3089e-19, 3.3089e-19, 3.3089e-19, 3.3089e-19, 3.3089e-19,
        3.3089e-19, 3.3089e-19, 3.3089e-19, 3.3089e-19])


In [ ]:

# x = ds.train_x[:100]
# y = ds.train_y[:100]
# embeds = model(x)
# loss = contrastive_loss(embeds,y)
# loss.backward()

In [92]:
# for i, j in contrastive_pairs:
#     embed1 = embeds[i]
#     embed2 = embeds[j]
#     print(embed1.shape, embed2.shape)
#     break

torch.Size([64]) torch.Size([64])


In [86]:
# embeds.shape, contrastive_pairs.shape

(torch.Size([100, 64]), torch.Size([1074, 2]))

In [85]:
# t.gather(embeds, dim=0, index=contrastive_pairs)

tensor([[ 0.0207, -0.0079],
        [ 0.0207, -0.1484],
        [ 0.0207, -0.2121],
        ...,
        [ 0.0572, -0.0880],
        [ 0.0572, -0.0749],
        [ 0.0572, -0.1148]], grad_fn=<GatherBackward0>)

In [66]:
# t.tensor(list(it.product(labels_sim_mat[0].nonzero().flatten(), repeat=2)))

tensor([[ 0,  0],
        [ 0, 11],
        [ 0, 35],
        [ 0, 47],
        [ 0, 65],
        [11,  0],
        [11, 11],
        [11, 35],
        [11, 47],
        [11, 65],
        [35,  0],
        [35, 11],
        [35, 35],
        [35, 47],
        [35, 65],
        [47,  0],
        [47, 11],
        [47, 35],
        [47, 47],
        [47, 65],
        [65,  0],
        [65, 11],
        [65, 35],
        [65, 47],
        [65, 65]])